In [ ]:
!pip install -q tensorflow-model-optimization

In [12]:
import tempfile
import os

import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing

In [13]:
# Load data
student_df = pd.read_csv('../../datasets/student_mat_processed01.csv')

train_dataset = student_df.sample(frac=0.8, random_state=0)
test_dataset = student_df.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_G1_targets = train_features.pop('G1')
train_G2_targets = train_features.pop('G2')
train_G3_targets = train_features.pop('G3')

test_G1_targets = test_features.pop('G1')
test_G2_targets = test_features.pop('G2')
test_G3_targets = test_features.pop('G3')

In [14]:
# Normalizer Layer
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))

In [15]:
# Define the model architecture.
def build_and_compile_model():
    model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(45)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(1)
    ])

    # Train the model
    model.compile(loss='mean_absolute_error', metrics=[tf.keras.metrics.MeanAbsoluteError()],
                  optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

In [16]:
fc_model = build_and_compile_model()
fc_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                1472      
_________________________________________________________________
dense_5 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 9         
Total params: 2,145
Trainable params: 2,145
Non-trainable params: 0
_________________________________________________________________


In [17]:
history = fc_model.fit(
    train_features, train_G3_targets,
    validation_split=0.2,
    verbose=1, epochs=100)

Epoch 1/100
8/8 [==============================] - 0s 20ms/step - loss: 9.5633 - mean_absolute_error: 9.5633 - val_loss: 10.0867 - val_mean_absolute_error: 10.0867
Epoch 2/100
8/8 [==============================] - 0s 3ms/step - loss: 8.7064 - mean_absolute_error: 8.7064 - val_loss: 9.3412 - val_mean_absolute_error: 9.3412
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 8.0079 - mean_absolute_error: 8.0079 - val_loss: 8.5286 - val_mean_absolute_error: 8.5286
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: 7.2579 - mean_absolute_error: 7.2579 - val_loss: 7.5121 - val_mean_absolute_error: 7.5121
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 6.4111 - mean_absolute_error: 6.4111 - val_loss: 6.3569 - val_mean_absolute_error: 6.3569
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: 5.3992 - mean_absolute_error: 5.3992 - val_loss: 4.8502 - val_mean_absolute_error: 4.8502
Epoch 7/100
8/8 [==============

8/8 [==============================] - 0s 3ms/step - loss: 2.8988 - mean_absolute_error: 2.8988 - val_loss: 3.1594 - val_mean_absolute_error: 3.1594
Epoch 52/100
8/8 [==============================] - 0s 3ms/step - loss: 2.8763 - mean_absolute_error: 2.8763 - val_loss: 3.2833 - val_mean_absolute_error: 3.2833
Epoch 53/100
8/8 [==============================] - 0s 4ms/step - loss: 2.8545 - mean_absolute_error: 2.8545 - val_loss: 3.2033 - val_mean_absolute_error: 3.2033
Epoch 54/100
8/8 [==============================] - 0s 3ms/step - loss: 2.8842 - mean_absolute_error: 2.8842 - val_loss: 3.1974 - val_mean_absolute_error: 3.1974
Epoch 55/100
8/8 [==============================] - 0s 3ms/step - loss: 2.8315 - mean_absolute_error: 2.8315 - val_loss: 3.2912 - val_mean_absolute_error: 3.2912
Epoch 56/100
8/8 [==============================] - 0s 3ms/step - loss: 2.9192 - mean_absolute_error: 2.9192 - val_loss: 3.0420 - val_mean_absolute_error: 3.0420
Epoch 57/100
8/8 [=======================

In [19]:
test_results = {}
_, test_results['fc_model'] = fc_model.evaluate(test_features, test_G3_targets, verbose=0)

print('Baseline mean absolute error:', test_results['fc_model'])

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(fc_model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline mean absolute error: 3.327263593673706
Saved baseline model to: C:\Users\z0042fkb\AppData\Local\Temp\tmpti92je5d.h5


In [20]:
import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

In [21]:
# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 8
validation_split = 0.1 # 10% of training set will be used for validation set.

num_features = train_features.shape[0] * (1 - validation_split)
end_step = np.ceil(num_features / batch_size).astype(np.int32) * epochs

In [22]:
# Define model for pruning.
pruning_params = {
    'pruning_schedule':
    tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                         final_sparsity=0.80,
                                         begin_step=0,
                                         end_step=end_step)
}

model_for_pruning = prune_low_magnitude(fc_model, **pruning_params)

In [23]:
# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mean_absolute_error',
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

model_for_pruning.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_dense_4  (None, 32)                2914      
_________________________________________________________________
prune_low_magnitude_dense_5  (None, 16)                1042      
_________________________________________________________________
prune_low_magnitude_dense_6  (None, 8)                 266       
_________________________________________________________________
prune_low_magnitude_dense_7  (None, 1)                 19        
Total params: 4,241
Trainable params: 2,145
Non-trainable params: 2,096
_________________________________________________________________


In [42]:
# Training and evaluate the model against baseline
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_features, train_G3_targets,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/8
2/9 [=====>........................] - ETA: 19s - loss: 6.9465 - mean_absolute_error: 6.9465WARNING:tensorflow:Callbacks method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.0030s vs `on_train_batch_begin` time: 0.0298s). Check your callbacks.


9/9 [==============================] - 6s 675ms/step - loss: 6.4324 - mean_absolute_error: 6.4324 - val_loss: 6.2029 - val_mean_absolute_error: 6.2029
Epoch 2/8
9/9 [==============================] - 0s 9ms/step - loss: 4.8711 - mean_absolute_error: 4.8711 - val_loss: 4.7083 - val_mean_absolute_error: 4.7083
Epoch 3/8
9/9 [==============================] - 0s 8ms/step - loss: 3.6748 - mean_absolute_error: 3.6748 - val_loss: 3.7654 - val_mean_absolute_error: 3.7654
Epoch 4/8
9/9 [==============================] - 0s 8ms/step - loss: 3.2219 - mean_absolute_error: 3.2219 - val_loss: 3.3748 - val_mean_absolute_error: 3.3748
Epoch 5/8
9/9 [==============================] - 0s 8ms/step - loss: 3.0859 - mean_absolute_error: 3.0859 - val_loss: 3.2134 - val_mean_absolute_error: 3.2134
Epoch 6/8
9/9 [==============================] - 0s 8ms/step - loss: 3.0631 - mean_absolute_error: 3.0631 - val_loss: 3.1850 - val_mean_absolute_error: 3.1850
Epoch 7/8
9/9 [==============================] - 0s 8m

In [43]:
_, test_results['pruned_model'] = model_for_pruning.evaluate(
   test_features, test_G3_targets, verbose=1)

print('Baseline mean absolute error:', test_results['fc_model']) 
print('Pruned mean absolute error:', test_results['pruned_model'])

3/3 [==============================] - 0s 4ms/step - loss: 3.2374 - mean_absolute_error: 3.2374
Baseline mean absolute error: 3.30635404586792
Pruned mean absolute error: 3.2373874187469482


In [44]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export,
                           pruned_keras_file,
                           include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: C:\Users\z0042fkb\AppData\Local\Temp\tmpmzejx93k.h5


In [45]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

INFO:tensorflow:Assets written to: C:\Users\z0042fkb\AppData\Local\Temp\tmpkahc1xcp\assets


INFO:tensorflow:Assets written to: C:\Users\z0042fkb\AppData\Local\Temp\tmpkahc1xcp\assets


Saved pruned TFLite model to: C:\Users\z0042fkb\AppData\Local\Temp\tmp1dqzg5j3.tflite


In [46]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [47]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 9872.00 bytes
Size of gzipped pruned Keras model: 6805.00 bytes
Size of gzipped pruned TFlite model: 5982.00 bytes


In [39]:
#!pip install -U tensorboard
#%load_ext tensorboard
#%tensorboard --logdir={logdir}
#print(logdir)
#%tensorboard dev upload --logdir 'C:\\Users\\z0042fkb\\AppData\\Local\\Temp\\tmppg6etlnp'
#%tensorboard dev auth revoke

ERROR: Failed to launch TensorBoard (exited with 0).
Contents of stderr:
2021-04-27 17:50:53.881608: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2021-04-27 17:50:53.890311: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Logged out of uploader.